In [1]:
# Reference: https://pytorch.org/hub/ultralytics_yolov5/
import torch
import os
import cv2
import numpy as np

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\uttej/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-10-15 Python-3.11.3 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
test_dir = 'D:\\present\\cmpe249-hw1\\dataset\\images\\test'
images = os.listdir(test_dir)

image_paths = []
for img_name in images:
	image_paths.append(test_dir + '\\' + img_name)

images = [cv2.imread(img_path) for img_path in image_paths]

In [4]:
# Get predictions
results = model(images)

In [5]:
# Store predictions in list of list of lists
df_lists = results.pandas().xyxy

all_detections = []
for df in df_lists:
	all_detections.append(df.values.tolist())

In [6]:
# Only select those with required categories
# Convert keys to lowercase to match with detections' categories
categories = {
    "pedestrian": 0,
    "truck": 1,
    "car": 2,
    "cyclist": 3,
    "dontCare": 4,
    "misc": 5,
    "van": 6,
    "tram": 7,
    "person_sitting": 8,
}

In [7]:
tmp_image_path = 'D:\\present\\cmpe249-hw1\\dataset\\images\\test\\001000.png'
img_width = cv2.imread(tmp_image_path).shape[1]
img_height = cv2.imread(tmp_image_path).shape[0]

In [8]:
image_id = 0
coco_results = []
for image_detections in all_detections:
	image_id += 1
	for detection in image_detections:
		x1, y1, x2, y2, conf, clss, category = detection

		intx1 = int(x1)
		inty1 = int(y1)
		intx2 = int(x2)
		inty2 = int(y2)

		bbox_center_x = float( (x1 + (x2 - x1) / 2.0) / img_width)
		bbox_center_y = float( (y1 + (y2 - y1) / 2.0) / img_height)
		bbox_width = float((x2 - x1) / img_width)
		bbox_height = float((y2 - y1) / img_height)

		if category in categories:
			coco_result = {
				"image_id": image_id,
				"category_id": categories[category],
				"bbox": [bbox_center_x, bbox_center_y, bbox_width, bbox_height],
				"score": conf,
			}
			coco_results.append(coco_result)

In [9]:
import json

with open('predictions.json', 'w') as f:
	json.dump(coco_results, f)